In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install autogluon

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv', index_col='PassengerId')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv', index_col='PassengerId')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
submission_id = test.index

In [ ]:
#dividing Cabin into three parts
train[['Cabin1', 'Cabin2', 'Cabin3']] = train['Cabin'].str.split('/', expand=True)
test[['Cabin1', 'Cabin2', 'Cabin3']] = test['Cabin'].str.split('/', expand=True)
train.drop(columns='Cabin', inplace=True)
test.drop(columns='Cabin', inplace=True)
train['Cabin2'] = train['Cabin2'].astype(float)
test['Cabin2'] = test['Cabin2'].astype(float)

In [ ]:
pip install gender_guesser

In [ ]:
#name itself is probably not important, but gender could be important
import gender_guesser.detector as gender
d = gender.Detector()
train['name_to_predict'] = train['Name'].map(lambda x: str(x).split(' ')[0])
test['name_to_predict'] = test['Name'].map(lambda x: str(x).split(' ')[0])
train['gender'] = train['name_to_predict'].map(lambda x: d.get_gender(x))
test['gender'] = test['name_to_predict'].map(lambda x: d.get_gender(x))
train.drop(columns=['Name','name_to_predict'], inplace=True)
test.drop(columns=['Name','name_to_predict'], inplace=True)

In [ ]:
#mostly_male -> male, mostly_female -> female
train.loc[train['gender']=='mostly_male', 'gender'] = 'male'
test.loc[test['gender']=='mostly_male', 'gender'] = 'male'
train.loc[train['gender']=='mostly_female', 'gender'] = 'female'
test.loc[test['gender']=='mostly_female', 'gender'] = 'female'

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

label = 'Transported'  
eval_metric = 'accuracy'

predictor = TabularPredictor(label=label, eval_metric=eval_metric, verbosity=3).fit(
    train, presets='best_quality', time_limit=3600)
results = predictor.fit_summary()

In [ ]:
y_pred = predictor.predict(test)

In [ ]:
y_pred.head()

In [ ]:
submission = pd.DataFrame({'PassengerId':submission_id, 'Transported': y_pred.values})
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()